# Advanced SQL with Python using sqlite3

This notebook continues from the introductory SQL notebook and introduces more advanced SQL concepts including multiple tables, foreign keys, joins, aggregations, and subqueries. We'll work with a school database that has students, courses, and enrollments.

## Import Required Libraries and Connect to Database

We'll import sqlite3 and connect to a new database file called 'school.db'.

In [1]:
import sqlite3

# Connect to the school database
conn = sqlite3.connect('school.db')
cursor = conn.cursor()

print("Connected to school database successfully!")

Connected to school database successfully!


## Creating Related Tables with Foreign Keys

We'll create three tables:
- `students`: Basic student information
- `courses`: Course details
- `enrollments`: Links students to courses with grades (many-to-many relationship)

Foreign keys ensure referential integrity - you can't enroll a non-existent student in a course.

In [2]:
# Create students table
cursor.execute('''
CREATE TABLE IF NOT EXISTS students (
    student_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT UNIQUE,
    enrollment_year INTEGER
)
''')

# Create courses table
cursor.execute('''
CREATE TABLE IF NOT EXISTS courses (
    course_id INTEGER PRIMARY KEY,
    course_name TEXT NOT NULL,
    instructor TEXT,
    credits INTEGER
)
''')

# Create enrollments table with foreign keys
cursor.execute('''
CREATE TABLE IF NOT EXISTS enrollments (
    enrollment_id INTEGER PRIMARY KEY,
    student_id INTEGER,
    course_id INTEGER,
    grade TEXT,
    enrollment_date DATE,
    FOREIGN KEY (student_id) REFERENCES students (student_id),
    FOREIGN KEY (course_id) REFERENCES courses (course_id)
)
''')

print("Tables created successfully with foreign key relationships!")

Tables created successfully with foreign key relationships!


## Inserting Data into Related Tables

We'll add sample data to all three tables, maintaining referential integrity.

In [3]:
# Insert students
students_data = [
    (1, 'Alice Johnson', 'alice@email.com', 2023),
    (2, 'Bob Smith', 'bob@email.com', 2023),
    (3, 'Charlie Brown', 'charlie@email.com', 2022),
    (4, 'Diana Prince', 'diana@email.com', 2024)
]
cursor.executemany("INSERT OR IGNORE INTO students VALUES (?, ?, ?, ?)", students_data)

# Insert courses
courses_data = [
    (1, 'Introduction to Python', 'Dr. Smith', 3),
    (2, 'Database Systems', 'Prof. Johnson', 4),
    (3, 'Web Development', 'Dr. Davis', 3),
    (4, 'Data Structures', 'Prof. Wilson', 4)
]
cursor.executemany("INSERT OR IGNORE INTO courses VALUES (?, ?, ?, ?)", courses_data)

# Insert enrollments
enrollments_data = [
    (1, 1, 1, 'A', '2023-09-01'),
    (2, 1, 2, 'B+', '2023-09-01'),
    (3, 2, 1, 'A-', '2023-09-01'),
    (4, 2, 3, 'B', '2023-09-01'),
    (5, 3, 2, 'A', '2022-09-01'),
    (6, 3, 4, 'B+', '2022-09-01'),
    (7, 4, 1, 'A', '2024-09-01'),
    (8, 4, 3, 'A-', '2024-09-01')
]
cursor.executemany("INSERT OR IGNORE INTO enrollments VALUES (?, ?, ?, ?, ?)", enrollments_data)

conn.commit()
print("Sample data inserted successfully!")

Sample data inserted successfully!


## INNER JOIN - Combining Data from Multiple Tables

An INNER JOIN returns only rows where there is a match in both tables. Let's see which students are enrolled in which courses.

In [4]:
# INNER JOIN to show student enrollments with course details
cursor.execute('''
SELECT s.name, c.course_name, e.grade, e.enrollment_date
FROM students s
INNER JOIN enrollments e ON s.student_id = e.student_id
INNER JOIN courses c ON e.course_id = c.course_id
ORDER BY s.name, c.course_name
''')

results = cursor.fetchall()
print("Student Enrollments (INNER JOIN):")
print("Name | Course | Grade | Date")
print("-" * 40)
for row in results:
    print(f"{row[0]} | {row[1]} | {row[2]} | {row[3]}")

Student Enrollments (INNER JOIN):
Name | Course | Grade | Date
----------------------------------------
Alice Johnson | Database Systems | B+ | 2023-09-01
Alice Johnson | Introduction to Python | A | 2023-09-01
Bob Smith | Introduction to Python | A- | 2023-09-01
Bob Smith | Web Development | B | 2023-09-01
Charlie Brown | Data Structures | B+ | 2022-09-01
Charlie Brown | Database Systems | A | 2022-09-01
Diana Prince | Introduction to Python | A | 2024-09-01
Diana Prince | Web Development | A- | 2024-09-01


## LEFT JOIN - Including All Records from Left Table

A LEFT JOIN returns all rows from the left table and matching rows from the right table. If no match, NULL values are returned. This is useful to find students who haven't enrolled in any courses.

In [5]:
# Add a student with no enrollments to demonstrate LEFT JOIN
cursor.execute("INSERT OR IGNORE INTO students VALUES (?, ?, ?, ?)", (5, 'Eve Wilson', 'eve@email.com', 2024))
conn.commit()

# LEFT JOIN to show all students and their enrollments (including those with none)
cursor.execute('''
SELECT s.name, c.course_name, e.grade
FROM students s
LEFT JOIN enrollments e ON s.student_id = e.student_id
LEFT JOIN courses c ON e.course_id = c.course_id
ORDER BY s.name
''')

results = cursor.fetchall()
print("All Students and Their Enrollments (LEFT JOIN):")
print("Name | Course | Grade")
print("-" * 30)
for row in results:
    course = row[1] if row[1] else "No enrollment"
    grade = row[2] if row[2] else "N/A"
    print(f"{row[0]} | {course} | {grade}")

All Students and Their Enrollments (LEFT JOIN):
Name | Course | Grade
------------------------------
Alice Johnson | Introduction to Python | A
Alice Johnson | Database Systems | B+
Bob Smith | Introduction to Python | A-
Bob Smith | Web Development | B
Charlie Brown | Database Systems | A
Charlie Brown | Data Structures | B+
Diana Prince | Introduction to Python | A
Diana Prince | Web Development | A-
Eve Wilson | No enrollment | N/A


## Aggregation with GROUP BY

GROUP BY groups rows that have the same values in specified columns. Combined with aggregate functions like COUNT(), AVG(), SUM(), etc., it provides summary information.

In [6]:
# Count enrollments per student
cursor.execute('''
SELECT s.name, COUNT(e.enrollment_id) as courses_taken
FROM students s
LEFT JOIN enrollments e ON s.student_id = e.student_id
GROUP BY s.student_id, s.name
ORDER BY courses_taken DESC
''')

results = cursor.fetchall()
print("Courses Taken by Each Student:")
for row in results:
    print(f"{row[0]}: {row[1]} courses")

print()

# Average credits per enrollment year
cursor.execute('''
SELECT s.enrollment_year, AVG(c.credits) as avg_credits
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
JOIN courses c ON e.course_id = c.course_id
GROUP BY s.enrollment_year
ORDER BY s.enrollment_year
''')

results = cursor.fetchall()
print("Average Credits by Enrollment Year:")
for row in results:
    print(f"Year {row[0]}: {row[1]:.2f} credits")

Courses Taken by Each Student:
Alice Johnson: 2 courses
Bob Smith: 2 courses
Charlie Brown: 2 courses
Diana Prince: 2 courses
Eve Wilson: 0 courses

Average Credits by Enrollment Year:
Year 2022: 4.00 credits
Year 2023: 3.25 credits
Year 2024: 3.00 credits


## Subqueries - Queries Within Queries

Subqueries allow you to nest one query inside another. They can be used in SELECT, FROM, WHERE, and other clauses.

In [7]:
# Find students enrolled in the most popular course
cursor.execute('''
SELECT s.name, c.course_name
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
JOIN courses c ON e.course_id = c.course_id
WHERE c.course_id = (
    SELECT course_id
    FROM enrollments
    GROUP BY course_id
    ORDER BY COUNT(*) DESC
    LIMIT 1
)
''')

results = cursor.fetchall()
print("Students in the Most Popular Course:")
for row in results:
    print(f"{row[0]} - {row[1]}")

print()

# Students with above-average number of enrollments
cursor.execute('''
SELECT s.name, COUNT(e.enrollment_id) as courses_taken
FROM students s
LEFT JOIN enrollments e ON s.student_id = e.student_id
GROUP BY s.student_id, s.name
HAVING courses_taken > (
    SELECT AVG(course_count)
    FROM (
        SELECT COUNT(enrollment_id) as course_count
        FROM enrollments
        GROUP BY student_id
    )
)
ORDER BY courses_taken DESC
''')

results = cursor.fetchall()
print("Students with Above-Average Course Load:")
for row in results:
    print(f"{row[0]}: {row[1]} courses")

Students in the Most Popular Course:
Alice Johnson - Introduction to Python
Bob Smith - Introduction to Python
Diana Prince - Introduction to Python

Students with Above-Average Course Load:


## Advanced Operations - UPDATE with JOIN

You can update records based on data from other tables using subqueries or joins in the WHERE clause.

In [8]:
# Update grades for students in their first year (enrollment_year = 2024)
cursor.execute('''
UPDATE enrollments
SET grade = 'A+'
WHERE student_id IN (
    SELECT student_id
    FROM students
    WHERE enrollment_year = 2024
)
''')

conn.commit()
print("Updated grades for first-year students to A+")

# Verify the update
cursor.execute('''
SELECT s.name, s.enrollment_year, e.grade
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
WHERE s.enrollment_year = 2024
''')

results = cursor.fetchall()
print("First-year students' grades after update:")
for row in results:
    print(f"{row[0]} (Year {row[1]}): {row[2]}")

Updated grades for first-year students to A+
First-year students' grades after update:
Diana Prince (Year 2024): A+
Diana Prince (Year 2024): A+


## Close the Database Connection

Always remember to close your database connections when done.

In [ ]:
# Close the database connection
conn.close()

print("Database connection closed successfully!")